In [1]:
# Start Main Code To Make Model

In [2]:
from pickle import dump

In [3]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = "2"

In [4]:
imgs_source_path = './images/renamed_train/'

In [5]:
all_imgs_list = os.listdir(imgs_source_path)

In [6]:
num_train_imgs = 20
num_test_imgs = 5

In [7]:
class_list = list(set([x.split('.')[-3] for x in all_imgs_list]))
# content = '\t'
# for c in class_list:
#     content += c
#     if c != class_list[-1]:
#         content += ' '
class_list_path = 'class_list.txt'
file = open(class_list_path, 'w')
file.write(' '.join(class_list))
file.close()

In [8]:
import random

In [9]:
train_imgs_list = list()
test_imgs_list = list()

for i in range(len(class_list)):
    class_name = class_list[i]
    class_all_imgs_list = [x for x in all_imgs_list if x.split('.')[-3] == class_name]
    class_train_imgs_list = list()
    class_test_imgs_list = list()
    for j in range(num_train_imgs):
        img_name = random.choice(class_all_imgs_list)
        if img_name not in class_train_imgs_list:
            class_train_imgs_list.append(img_name)
            class_all_imgs_list.remove(img_name)
    for j in range(num_test_imgs):
        img_name = random.choice(class_all_imgs_list)
        if img_name not in class_train_imgs_list and img_name not in class_test_imgs_list:
            class_test_imgs_list.append(img_name)
            class_all_imgs_list.remove(img_name)
    train_imgs_list += class_train_imgs_list
    test_imgs_list += class_test_imgs_list
# train_imgs_list, test_imgs_list


In [10]:
img_size = 300

In [11]:
import tensorflow as tf
tf.get_logger().setLevel('ERROR')

In [12]:
def preprocess_image(img):
    img = tf.image.decode_png(img, channels=3)
    img = tf.image.resize(img, [img_size, img_size])
    return img

In [13]:
def load_and_preprocess_image(path):
    img = tf.io.read_file(imgs_source_path + path)
    return preprocess_image(img)

In [14]:
map_dic = { class_list[i]: i for i in range(len(class_list)) }

In [15]:
X_train = train_imgs_list
X_val = test_imgs_list
Y_train = [map_dic[x.split('.')[-3]] for x in X_train]
Y_val = [map_dic[x.split('.')[-3]] for x in X_val]

In [16]:
DS_train = tf.data.Dataset.from_tensor_slices((X_train, Y_train))
DS_val = tf.data.Dataset.from_tensor_slices((X_val, Y_val))

In [17]:
def load_and_preprocess_from_path_label(path, label):
    return load_and_preprocess_image(path), tf.one_hot(label, len(class_list))

In [18]:
DS_train = DS_train.map(load_and_preprocess_from_path_label)
DS_val = DS_val.map(load_and_preprocess_from_path_label)

In [19]:
len(DS_train), len(DS_val)

(60, 15)

In [20]:
batch_size=4

In [21]:
DS_train = DS_train.shuffle(buffer_size=int(len(DS_train) * 0.8))

In [22]:
DS_batch_train = DS_train.batch(batch_size=batch_size, drop_remainder=False)
DS_batch_train = DS_batch_train.prefetch(tf.data.AUTOTUNE)
DS_batch_val = DS_val.batch(batch_size=batch_size, drop_remainder=False)

In [23]:
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers
import numpy as np
import matplotlib.pyplot as plt

In [24]:
augmentation = Sequential([
#     layers.RandomBrightness(factor=(-0.15, 0.15)),
    layers.RandomRotation(factor=(-0.15, 0.15)),
    layers.RandomTranslation(height_factor=0.1, width_factor=0.1),
    layers.RandomFlip(),
    layers.RandomContrast(factor=0.1),
], name='augmentation',)

In [25]:
n_classes = len(class_list)

In [26]:
from tensorflow.keras.applications.efficientnet import EfficientNetB3
from tensorflow.keras.applications.vgg19 import VGG19

In [27]:
def build_model(n_classes):
    model = Sequential()
    model.add(layers.Input(shape=(img_size, img_size, 3)))
    model.add(augmentation)
    
    base_model = VGG19(include_top=False, weights=None)
    base_model.trainable = True
    model.add(base_model)
    
    model.add(layers.GlobalAveragePooling2D())
    model.add(layers.BatchNormalization())
    model.add(layers.Dropout(0.5))
    model.add(layers.Dense(n_classes, activation='softmax'))
    
    optimizer = tf.keras.optimizers.Adam(learning_rate=1e-4)
    model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
    
    return model

In [28]:
model = build_model(n_classes)

In [29]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 augmentation (Sequential)   (None, 300, 300, 3)       0         
                                                                 
 vgg19 (Functional)          (None, None, None, 512)   20024384  
                                                                 
 global_average_pooling2d (G  (None, 512)              0         
 lobalAveragePooling2D)                                          
                                                                 
 batch_normalization (BatchN  (None, 512)              2048      
 ormalization)                                                   
                                                                 
 dropout (Dropout)           (None, 512)               0         
                                                                 
 dense (Dense)               (None, 3)                 1

In [30]:
epochs = 10

In [31]:
from tensorflow.keras.callbacks import ReduceLROnPlateau

In [32]:
learning_rate_reduction = ReduceLROnPlateau(
    monitor='loss',
    lr=1e-4,
    patience=2,
    verbose=1,
    factor=0.1,
    min_lr=1e-10,
)

In [33]:
callbacks = [learning_rate_reduction]

In [34]:
try:
    hist = model.fit(
        DS_batch_train, 
        epochs=epochs, 
        validation_data=DS_batch_val, 
        batch_size=batch_size, 
        shuffle=True, 
        verbose=1, 
        callbacks=callbacks
    )
except Exception as e:
    print(e)
    pass

Epoch 1/10
15/15 [==============================] - 51s 660ms/step - loss: 1.2355 - accuracy: 0.4833 - val_loss: 3.3946 - val_accuracy: 0.3333 - lr: 1.0000e-04
Epoch 2/10
15/15 [==============================] - 13s 505ms/step - loss: 0.6093 - accuracy: 0.7667 - val_loss: 11.0669 - val_accuracy: 0.3333 - lr: 1.0000e-04
Epoch 3/10
15/15 [==============================] - 13s 510ms/step - loss: 0.6391 - accuracy: 0.7500 - val_loss: 5.3125 - val_accuracy: 0.3333 - lr: 1.0000e-04
Epoch 4/10
15/15 [==============================] - ETA: 0s - loss: 0.6110 - accuracy: 0.7833
Epoch 4: ReduceLROnPlateau reducing learning rate to 9.999999747378752e-06.
15/15 [==============================] - 13s 528ms/step - loss: 0.6110 - accuracy: 0.7833 - val_loss: 15.9210 - val_accuracy: 0.4000 - lr: 1.0000e-04
Epoch 5/10
15/15 [==============================] - 14s 509ms/step - loss: 0.6654 - accuracy: 0.7167 - val_loss: 5.4895 - val_accuracy: 0.6000 - lr: 1.0000e-05
Epoch 6/10
15/15 [=====================

In [37]:
for i in range(10):
    img_test_path = random.choice(all_imgs_list)
    print(img_test_path)
    DS_test = tf.data.Dataset.from_tensor_slices([img_test_path])
    DS_test = DS_test.map(load_and_preprocess_image)
    DS_batch_test = DS_test.batch(batch_size=batch_size, drop_remainder=False)
    result = model.predict(DS_batch_test, batch_size=batch_size, max_queue_size=1, verbose=0)
    print(result)
    print(class_list[np.argmax(result[0])], '\n')

dinh_lang.32.png
[[1.1411355e-03 8.8321010e-04 9.9797565e-01]]
dinh_lang 

dinh_lang.19.png
[[1.6716119e-03 9.4919628e-04 9.9737918e-01]]
dinh_lang 

dinh_lang.12.png
[[0.00107792 0.00309768 0.99582446]]
dinh_lang 

tia_to.22.png
[[2.2989327e-04 9.9513853e-01 4.6315594e-03]]
tia_to 

dinh_lang.26.png
[[7.4383128e-04 2.0071524e-03 9.9724895e-01]]
dinh_lang 

co_man_trau.14.png
[[0.9168083  0.01514977 0.06804196]]
co_man_trau 

co_man_trau.9.png
[[0.72476953 0.0145345  0.26069593]]
co_man_trau 

dinh_lang.22.png
[[0.00123198 0.00103496 0.997733  ]]
dinh_lang 

co_man_trau.38.png
[[0.23076989 0.02317927 0.7460509 ]]
dinh_lang 

tia_to.8.png
[[7.192879e-05 9.993556e-01 5.723943e-04]]
tia_to 



In [36]:
model.save('model.h5')